In [1]:
import pandas as pd
import numpy as np
import os, gc
os.chdir("/home/jovyan/work/phd/datasets/cancers/lung")

In [2]:
df=pd.read_csv("mainTable.csv", index_col=0)
df.info()
df.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 3000 entries, ENSG00000000938 to ENSG00000273474
Columns: 408 entries, dc0ce32f-d5aa-489f-ba4b-e01443815121.FPKM.txt.gz to b1558748-bec9-4bea-a35b-040ca9a1f4cd.FPKM.txt.gz
dtypes: float64(408)
memory usage: 9.4+ MB


,dc0ce32f-d5aa-489f-ba4b-e01443815121.FPKM.txt.gz,2b33014e-c7ff-41aa-82bf-3121756639ad.FPKM.txt.gz,397e302e-393e-4f54-b360-fcffe394271e.FPKM.txt.gz,037b5a94-487e-4f5b-90a3-d696eb4a8a3e.FPKM.txt.gz,5e010a9a-7f65-434b-ae4c-43948461187c.FPKM.txt.gz,ae59d924-06ec-428d-b470-e78ed92c5e36.FPKM.txt.gz,2b2b6341-1f6d-4acc-8a67-0ba8ab051ee8.FPKM.txt.gz,87d1452e-8ec6-4c75-a03b-361738c728c0.FPKM.txt.gz,5d420937-4d97-4f04-9ce9-754054261d97.FPKM.txt.gz,c622f68c-e37c-45ee-a266-ae704629f0f6.FPKM.txt.gz,...,90af0faf-3e8c-4741-bf49-f87a5b21d54c.FPKM.txt.gz,3b1e8e56-f2a3-4a3f-8f99-ba8a8f33152c.FPKM.txt.gz,26918c74-1269-4f63-a871-57855b4aa846.FPKM.txt.gz,ef6e0187-ca29-4c32-9731-4f9a6b51ff80.FPKM.txt.gz,c8fb3217-2cc9-47c0-84b6-b666c07624e6.FPKM.txt.gz,68acaa5e-6a27-499f-925d-710af9d30710.FPKM.txt.gz,ce786209-c77d-47e0-95a6-20c036bed8b3.FPKM.txt.gz,cf792f7f-a3ee-4544-b029-8c4e4da4f973.FPKM.txt.gz,85ac083c-dbb9-4541-8b75-d73ab216f54c.FPKM.txt.gz,b1558748-bec9-4bea-a35b-040ca9a1f4cd.FPKM.txt.gz
0,,,,,,,,,,,,,,,,,,,,,
ENSG00000000938,4.24,2.53,1.90,4.10,20.25,6.07,15.66,1.41,4.82,12.22,...,8.31,3.61,4.01,3.92,3.45,2.46,5.88,16.31,4.70,7.87
ENSG00000001084,11.76,85.19,40.94,0.96,3.06,63.92,3.58,3.87,1.73,17.58,...,58.75,0.83,38.74,7.36,2.11,3.61,9.72,5.45,11.11,13.07


In [3]:
with open("corpus.txt",'w') as file:
    for sample in df.columns:
        for g in np.array(df[sample].sort_values(ascending=False).index[:1000],dtype=str):
            file.write(g[:15])
            file.write(" ")
        file.write("\n")

In [4]:
gc.collect()

44

```bash
export PATH=$PATH:/home/jovyan/work/phd/topicmapping/bin
topicmap -f corpus.txt -r 10 -t 10 -seed 42 -o tm
```

In [7]:
os.system("export PATH=$PATH:/home/jovyan/work/phd/topicmapping/bin")
os.system("topicmap -f corpus.txt -r 10 -t 10 -seed 42 -o tm")

32512

In [8]:
df_words = pd.read_csv("tm/word_wn_count.txt", sep=' ', header=None)
df_words.columns=['word', 'word-id', 'occurrence']
df_words.sort_values('word-id')

,word,word-id,occurrence
2773,ENSG00000087086,0,408
2558,ENSG00000168878,1,383
2262,ENSG00000161055,2,299
2721,ENSG00000211895,3,406
2681,ENSG00000197249,4,403
...,...,...,...
184,ENSG00000099822,2811,1
185,ENSG00000169085,2812,1
186,ENSG00000005961,2813,1
187,ENSG00000257818,2814,1


In [9]:
df_topic_distr = pd.read_csv("tm/lda_gammas_final.txt", sep=' ', header=None)
df_topic_distr.columns=['Topic %d'%(t+1) for t in df_topic_distr.columns]
df_topic_distr.index.name='i_doc'
df_topic_distr.insert(0,'doc',df.columns)
df_topic_distr=df_topic_distr.dropna(how='all',axis=1)
df_topic_distr

,doc,Topic 1,Topic 2,Topic 3,Topic 4
i_doc,,,,,
0,dc0ce32f-d5aa-489f-ba4b-e01443815121.FPKM.txt.gz,0.054963,389.250000,155.345000,455.593000
1,2b33014e-c7ff-41aa-82bf-3121756639ad.FPKM.txt.gz,820.059000,0.043447,0.067419,180.073000
2,397e302e-393e-4f54-b360-fcffe394271e.FPKM.txt.gz,0.054963,0.043447,0.067419,1000.080000
3,037b5a94-487e-4f5b-90a3-d696eb4a8a3e.FPKM.txt.gz,0.054963,108.446000,0.067419,891.675000
4,5e010a9a-7f65-434b-ae4c-43948461187c.FPKM.txt.gz,0.054963,1000.040000,0.067419,0.077077
...,...,...,...,...,...
403,68acaa5e-6a27-499f-925d-710af9d30710.FPKM.txt.gz,0.054963,100.966000,495.844000,403.378000
404,ce786209-c77d-47e0-95a6-20c036bed8b3.FPKM.txt.gz,0.054963,0.043447,0.067419,1000.080000
405,cf792f7f-a3ee-4544-b029-8c4e4da4f973.FPKM.txt.gz,0.054963,0.043447,433.569000,566.576000


In [10]:
clusters = df_topic_distr.drop('doc',1).values.argmax(1)
df_clusters=pd.DataFrame()
for cluster in range(np.max(clusters)+1):
    elems=df.columns[clusters==cluster].values
    df_clusters.insert(0,'Cluster %d'%(cluster+1),np.concatenate([elems, ['' for _ in range(len(df.columns)- len(elems))]]))

In [11]:
df_clusters.sort_index(axis=1).to_csv("tm/tm_level_0_clusters.csv", index=False, header=True)

In [12]:
df_topic_distr.to_csv("tm/tm_level_0_topic-dist.csv", index=True, header=True)

In [13]:
df_word_distr = pd.DataFrame().fillna(0)
with open("tm/lda_betas_sparse_final.txt","r") as f:
    for line in f.read().split("\n"):
        row = line.split(" ")
        if len(row) < 2:
            continue
        topic = int(row[0])+1
        line=np.array(row[1:-1], dtype=float).reshape(int((len(row)-1)/2),2)
        for el in line:
            df_word_distr.at[df_words[df_words['word-id']==int(el[0])].word.values[0], f"Topic {topic}"] = el[1]
#df_word_distr.index=df_words['word']
df_word_distr.fillna(0)
df_word_distr.to_csv("tm/tm_level_0_word-dist.csv", index=True, header=True)

In [14]:
df_topics = pd.DataFrame()
max_L = df_word_distr.shape[0]
for topic in df_word_distr.columns[::-1]:
    t_series = df_word_distr[topic]
    t_series = t_series[t_series>t_series.quantile(0.99)]
    df_topics.insert(0,topic,np.concatenate((t_series.index.values,np.repeat(np.nan, df_word_distr.shape[0]-len(t_series)))))
df_topics.dropna(how="all", axis=0).to_csv("tm/tm_level_0_topics.csv", index=False, header=True)